In [ ]:
# default_exp learner

# learner

> API details.

In [ ]:
#export
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.tabular.data import *
from fastai.tabular.core import *
from fastai.tabular.model import *
from fastrenewables.core import *
from fastrenewables.data import *
from fastrenewables.model import *
from fastai.basics import *


In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
class RenewableLearner(Learner):
    "`Learner` for renewable data"
    def predict(self, ds_idx=1, test_dl=None, filter=True):
        device = next(learn.model.parameters()).device
        
        if test_dl is not None:
            cur_dl = test_dl
        elif ds_idx == 0:
            cur_dl = self.dls.train_ds
        elif ds_idx == 1:
            cur_dl = self.dls.valid_ds
            
        if type(cur_dl) == TabularPandas or \
            type(cur_dl) == TabDataLoader or \
            type(cur_dl) == TabularRenewables:
                
            cats = tensor(
                cur_dl.cats.values,
            ).long()
            xs = tensor(cur_dl.conts.values)
            targets = tensor(cur_dl.y.values)
        
        with torch.no_grad():
            preds = learn.model(cats.to(device), xs.to(device))
        
        preds, targets = to_np(preds).reshape(-1), to_np(targets).reshape(-1)
        if filter:
            preds[preds < 0] = 0
            preds[preds > 1.1] = 1.1
            
        return preds, targets

In [ ]:
show_doc(RenewableLearner, title_level=3)

<h3 id="RenewableLearner" class="doc_header"><code>class</code> <code>RenewableLearner</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>RenewableLearner</code>(**`dls`**, **`model`**, **`loss_func`**=*`None`*, **`opt_func`**=*`Adam`*, **`lr`**=*`0.001`*, **`splitter`**=*`trainable_params`*, **`cbs`**=*`None`*, **`metrics`**=*`None`*, **`path`**=*`None`*, **`model_dir`**=*`'models'`*, **`wd`**=*`None`*, **`wd_bn_bias`**=*`False`*, **`train_bn`**=*`True`*, **`moms`**=*`(0.95, 0.85, 0.95)`*) :: `Learner`

`Learner` for renewable data

In [ ]:
cont_names = ['T_HAG_2_M', 'RELHUM_HAG_2_M', 'PS_SFC_0_M', 'ASWDIFDS_SFC_0_M',
       'ASWDIRS_SFC_0_M', 'WindSpeed58m']
cat_names = ['TaskID', 'Month', 'Day', 'Hour']
pd.options.mode.chained_assignment=None
dls = RenewableDataLoaders.from_files(glob.glob("../data/*.h5"), y_names="PowerGeneration", 
                                      pre_procs=[FilterYear(year=2020), AddSeasonalFeatures], 
                                      cat_names=cat_names, cont_names=cont_names)

{'y_names': 'PowerGeneration', 'pre_procs': [FilterYear -- {}:
encodes: (object,object) -> encodes
decodes: , <class 'fastrenewables.core.AddSeasonalFeatures'>], 'cat_names': ['TaskID', 'Month', 'Day', 'Hour'], 'cont_names': ['T_HAG_2_M', 'RELHUM_HAG_2_M', 'PS_SFC_0_M', 'ASWDIFDS_SFC_0_M', 'ASWDIRS_SFC_0_M', 'WindSpeed58m']}


In [ ]:
#export
@delegates(Learner.__init__)
def renewable_learner(dls, layers=None, emb_szs=None, config=None, n_out=None, y_range=None, **kwargs):
    "Get a `Learner` using `dls`, with `metrics`, including a `TabularModel` created using the remaining params."
    if config is None: config = tabular_config()
    
    to = dls.train_ds
    emb_szs = get_emb_sz(dls.train_ds, {} if emb_szs is None else emb_szs)
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if layers is None: layers = [len(dls.cont_names), 200,100,n_out]
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
        
    embed_p = kwargs["embed_p"].pop() if "embed_p" in kwargs.keys() else 0.1
    
        
    if emb_szs is not None:
        emb_module = EmbeddingModule(None, embedding_dropout=embed_p, embedding_dimensions=emb_szs)
    print(emb_szs)
        
#     model = MultiLayerPerceptron(emb_szs, len(dls.cont_names), n_out, layers, y_range=y_range, **config)
    model = MultiLayerPerceptron(layers, embedding_module=emb_module, **config)
    
    return RenewableLearner(dls, model, **kwargs)




In [ ]:
rmse

In [ ]:
learn = renewable_learner(dls, metrics=rmse)

[(4, 3), (13, 7), (32, 11), (25, 10)]


In [ ]:
dls.show_batch()

,TaskID,Month,Day,Hour,T_HAG_2_M,RELHUM_HAG_2_M,PS_SFC_0_M,ASWDIFDS_SFC_0_M,ASWDIRS_SFC_0_M,WindSpeed58m,PowerGeneration
0,2,1,9,23,269.988,90.561000,93186.121000,37.436001,3.108996,4.762943,0.004
1,2,12,9,13,277.607,78.536000,92178.707001,29.986000,6.561998,11.545762,0.092
2,0,8,19,17,294.915,39.171999,100510.991998,99.365000,108.859000,5.201804,0.036
3,0,8,9,2,288.865,77.048000,99851.234012,90.366000,128.352001,1.575587,0.001
4,1,3,10,18,278.011,81.079000,97335.879012,54.759000,21.039999,6.971470,1.004
5,0,3,18,14,281.983,50.029999,100594.226998,56.228000,105.356999,8.746034,0.001
6,2,7,9,3,280.866,76.627000,93514.526993,110.078000,44.007998,2.567007,0.009
7,0,8,14,22,289.286,82.897000,100207.077999,89.198999,152.506998,2.194640,0.001
8,2,5,18,3,281.300,94.464000,92208.051045,95.578001,166.274000,1.089962,0.009
9,0,6,15,2,288.925,89.617000,99733.316007,123.950998,102.054000,1.253892,0.001


In [ ]:
learn.model

MultiLayerPerceptron(
  (embedding_module): EmbeddingModule(
    (embeddings): ModuleList(
      (0): Embedding(4, 3)
      (1): Embedding(13, 7)
      (2): Embedding(32, 11)
      (3): Embedding(25, 10)
    )
    (emb_drop): Dropout(p=0.1, inplace=False)
  )
  (final_activation): Identity()
  (embeds): ModuleList()
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=37, out_features=200, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=200, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=100, out_features=1, bias=True)
    )
  )
)

In [ ]:
learn.fit(1)

epoch,train_loss,valid_loss,_rmse,time
0,0.014606,0.013321,0.115415,00:02


In [ ]:
preds,target = learn.predict()

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_model.ipynb.
Converted 03_learner.ipynb.
Converted index.ipynb.
